In [28]:
!python /content/drive/MyDrive/Latent\ Transfer/Image\ Classifier/main.py

100% 225/225 [00:32<00:00,  6.96it/s]
Training Started:


100% 858/858 [05:54<00:00,  2.42it/s]
100% 225/225 [00:32<00:00,  6.92it/s]
Improvement-Detected, save-model
Epoch: 000 || Train Loss: 2.170 || Train Acc: 61.522 || Eval Loss: 1.126 || Eval Acc: 86.475


In [23]:
import os
import random
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from tqdm import tqdm
import time
import copy
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader

class Dataset(object):
    
    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __add__(self, other):
        return ConcatDataset([self, other])
        

class DatasetMNIST(Dataset):
    
    def __init__(self, file_path, transform=None):
        self.data = pd.read_csv(file_path)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):

        image = self.data.iloc[index, 1:].values.astype(np.uint8).reshape((28, 28))
        label = self.data.iloc[index, 0]
        
        if (label > 9):
          label = label - 1
        rgb = np.repeat(image[..., np.newaxis], 3, -1)
        if self.transform is not None:
            rgb = self.transform(rgb)
        return rgb, label



def prepare_dataloader(num_workers=8,
                       batch_size=1,
                      ):

    transform_train = transforms.Compose([transforms.ToPILImage(),
                                          transforms.Resize((224)),
                                          transforms.CenterCrop(224),
                                          transforms.ToTensor(),
                                          # transforms.Normalize(mean=(0.4346, 0.2213, 0.0783),
                                          #                      std=(0.2439, 0.1311, 0.0703))            
                                        ])
    

    transform_test = transforms.Compose([transforms.ToPILImage(),
                                         transforms.Resize((224)),
                                         transforms.CenterCrop(224),
                                         transforms.ToTensor(),
                                         #transforms.Normalize(mean=(0.4346, 0.2213, 0.0783),
                                         #                        std=(0.2439, 0.1311, 0.0703))
                                        ])
    
    train_data = DatasetMNIST('/content/drive/MyDrive/Latent Transfer/DomainB/DomainB_dataset/sign_mnist_train.csv', transform = transform_train)
    test_data = DatasetMNIST('/content/drive/MyDrive/Latent Transfer/DomainB/DomainB_dataset/sign_mnist_test.csv', transform = transform_test)
    train_loader = DataLoader(
                              train_data,
                              batch_size=batch_size,
                              shuffle=True
                             )
    
    test_loader = DataLoader(
                              test_data,
                              batch_size=batch_size,
                              shuffle=False
                             )

    return train_loader, test_loader, train_data, test_data

In [24]:
x, y, w, z = prepare_dataloader()

In [25]:
labels = []
for i in range(0, 27455):
  i, l = w[i]
  labels.append(l)

In [26]:
labelset = set(labels)
labelset

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23}